In [77]:
import pandas as pd
import numpy as np
import aggregation

In [78]:
# open elections_with_tracts.xlsx
elections_with_tracts = pd.read_excel('elections_with_tracts.xlsx')
# open acs_data.csv
acs_data = pd.read_csv('acs_data.csv')
# remove all columns ending in EA
acs_data = acs_data.loc[:, ~acs_data.columns.str.endswith('EA')]
# replace all (X), -, or N with nan
acs_data = acs_data.replace('(X)', np.nan)
acs_data = acs_data.replace('-', np.nan)
acs_data = acs_data.replace('N', np.nan)
# convert all columns except the first 2 to floats
for col in acs_data.columns[2:]:
    acs_data[col] = acs_data[col].astype(float)
    # replace all negative values with NaN except the first 4 columns
    acs_data[col] = acs_data[col].apply(lambda x: np.nan if x < 0 else x)

/var/folders/52/t8__xqw525g96s6z30kmvnfr0000gp/T/ipykernel_21085/1159155937.py:4: DtypeWarning: Columns (3,21,23,30,33,48,60,61,64,71,88,134,154,155,168,172,182,205,206,216,218,221,222,246,252,263,264,268,274,277,282,288,291,293,295,304,307,308,314,323,337,350,381,386,415,456,462,488,544,548,557,560,562,567,578,579,581,600,610,611,630,635,645,648,655,664,678,684,699,704,709,710,724,732,736,744,750) have mixed types. Specify dtype option on import or set low_memory=False.
  acs_data = pd.read_csv('acs_data.csv')


In [79]:
# open variables/variables_acs5_2022.csv
variables = pd.read_csv('variables/variables_acs5_2022.csv')

percent_columns = variables[variables['Label'].str.lower().str.contains(r'\brate\b|\bpercentage\b')]
percent_columns['Name'].apply(lambda x: x[:-2])

# for each row in acs_data
for index, row in acs_data.iterrows():
    # for each column in percent_columns
    for col in percent_columns['Name']:
        # if col + E and col + PE are in acs columns
        if col + 'E' in acs_data.columns and col + 'PE' in acs_data.columns:
            # get col + E and col + PE
            est = row[col + 'E']
            percent_est = row[col + 'PE']

            # if percent_est is nan, set PE to est and set E to nan
            if np.isnan(percent_est) or percent_est == 0:
                acs_data.at[index, col + 'PE'] = est
                acs_data.at[index, col + 'E'] = np.nan

In [80]:
# checking for columns with invalid data
invalid = []
# for each column except the first 2
for col in acs_data.columns[2:]:
    valid = False
    # for each value in the column
    for val in acs_data[col]:
        # if the value is not nan and is greater than 0
        if not np.isnan(val) and val > 0:
            valid = True
            break
    if not valid:
        invalid.append(col)
# remove invalid columns
acs_data = acs_data.drop(columns=invalid)

In [81]:
# open statistics.xlsx
statistics = pd.read_excel('statistics.xlsx')
statistics

,agg_var,type,pop_var
0,DP03_0119PE,percentage,DP03_0075E
1,DP03_0120PE,percentage,DP02_0014E
2,DP03_0121PE,percentage,NaN
3,DP03_0122PE,percentage,DP02_0002E
4,DP03_0123PE,percentage,DP02_0003E
5,DP03_0124PE,percentage,NaN
6,DP03_0125PE,percentage,DP02_0010E
7,DP03_0126PE,percentage,DP02_0011E
8,DP03_0127PE,percentage,NaN
9,DP03_0128PE,percentage,DP05_0001E


In [82]:
# for each column in acs_data
for col in acs_data.columns:
   # if it ends in E and not PE
   if col in statistics['agg_var'].values:
      # if pop_var is NaN, delete col from acs_data
      if statistics[statistics['agg_var'] == col]['pop_var'].isnull().values[0]:
         acs_data = acs_data.drop(columns=[col])
      elif col.endswith('PE') and (col[:-2] + 'E') in acs_data.columns:
         # drop the corresponding E column
         acs_data = acs_data.drop(columns=[col[:-2] + 'E'])
   # if it ends in E and PE in columns
   if col.endswith('E') and (col[:-1] + 'PE') not in statistics['agg_var'].values and (col[:-1] + 'PE') in acs_data.columns:
      # drop the corresponding PE column
      acs_data = acs_data.drop(columns=[col[:-1] + 'PE'])

acs_data

,geocode,year,DP02_0130E,DP03_0051E,DP03_0010E,DP03_0121E,DP02_0090E,DP05_0001E,DP02_0112E,DP03_0084E,...,DP02_0134E,DP05_0050E,DP02_0007E,DP05_0037E,DP02_0057E,DP03_0115E,DP02_0062E,DP03_0067E,DP05_0003E,DP03_0063E
0,06_001_422200,2010,13.0,1522.0,1506.0,NaN,1044.0,3127.0,861.0,103.0,...,377.0,0.0,14.0,0.0,447.0,318.0,228.0,15020.0,1763.0,79904.0
1,06_001_422400,2010,0.0,2171.0,1597.0,NaN,1095.0,4686.0,1734.0,0.0,...,235.0,0.0,0.0,0.0,2677.0,1425.0,353.0,14688.0,1691.0,66278.0
2,06_001_422300,2010,10.0,1591.0,1488.0,NaN,1198.0,3620.0,560.0,69.0,...,439.0,44.0,19.0,0.0,890.0,466.0,392.0,12681.0,1781.0,77234.0
3,06_001_420400,2010,8.0,1160.0,1238.0,NaN,487.0,3472.0,1997.0,44.0,...,40.0,0.0,8.0,0.0,1012.0,965.0,148.0,12903.0,1777.0,51303.0
4,06_001_421800,2010,0.0,897.0,1025.0,NaN,784.0,2162.0,268.0,88.0,...,153.0,0.0,13.0,0.0,145.0,213.0,177.0,18863.0,1180.0,110333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6497,02_170_000101,2022,28.0,270.0,194.0,NaN,499.0,512.0,505.0,10.0,...,0.0,0.0,7.0,460.0,10.0,72.0,200.0,18418.0,213.0,77947.0
6498,02_050_000200,2022,141.0,1937.0,2058.0,NaN,5919.0,6312.0,5765.0,200.0,...,0.0,0.0,22.0,1032.0,420.0,526.0,1622.0,18060.0,2878.0,118182.0
6499,50_007_003600,2023,896.0,2511.0,2624.0,NaN,4255.0,4990.0,4834.0,42.0,...,0.0,0.0,0.0,4156.0,98.0,374.0,673.0,19693.0,2928.0,78025.0
6500,50_007_000800,2023,687.0,1032.0,1130.0,NaN,2331.0,2430.0,2348.0,168.0,...,0.0,0.0,14.0,2393.0,69.0,193.0,180.0,27227.0,1276.0,156711.0


In [83]:
# open elections.xlsx
elections = pd.read_excel('elections.xlsx')

In [84]:
all_data = []

# create df called missing_data with columns filename, year, values, num_missing, num_total, missing_tracts
missing_data = pd.DataFrame(columns=['filename', 'year', 'variable', 'values', 'num_missing', 'num_total', 'prop_missing'])

for index, row in elections_with_tracts.iterrows():
    if index % 10 == 0:
        print(f'{index} of {len(elections_with_tracts)} rows processed')

    tracts = row['tracts'].split(',')
    
    # create percents dict 
    percent_dict = {tract: percent for tract, percent in zip(tracts, list(map(float, row['percents'].split(','))))}

    # year is year in row in elections where filename = row['filename']
    year = elections[elections['filename'] == row['filename']]['year'].values[0]
    
    # get rows where geocode is in tracts and year = year
    tract_rows = acs_data[(acs_data['geocode'].isin(tracts)) & (acs_data['year'] == year) & acs_data['DP05_0001E'] > 0]
    # get percents in order of the geocodes in tract_rows
    percents = pd.DataFrame({'percent': [percent_dict[geocode] for geocode in tract_rows['geocode']]})

    # concat
    tract_rows = pd.concat([tract_rows.reset_index(drop=True), percents], axis=1)

    election_data = {}

    for column in acs_data.columns[2:]:
        # get the values in tract_rows for column
        values = tract_rows[column].values
        valid_values = values[~np.isnan(values)]

        # if all values are NaN, continue
        if all(np.isnan(values)):
            election_data[column] = np.nan
        else:
            # if at least one but not all values are NaN
            if any(np.isnan(values)):
                # add to missing_data
                missing_data.loc[len(missing_data)] = [row['filename'], year, column, values, sum(np.isnan(values)), len(values), sum(np.isnan(values)) / len(values)]

            if column in statistics['agg_var'].values:
                pop_var = statistics[statistics['agg_var'] == column]['pop_var'].values[0]
                # if pop_var is none skip
                if pd.isnull(pop_var):
                    continue

                # if pop_var contains -
                if '-' in pop_var:
                    # split on -
                    pop_vars = pop_var.split('-')
                    pops = tract_rows[pop_vars[0]].values - tract_rows[pop_vars[1]].values
                else:
                    pops = tract_rows[pop_var].values
                
                percents = tract_rows['percent'].values
                tract_pops = pops*percents
                valid_tract_pops = tract_pops[~np.isnan(values)]

                if statistics[statistics['agg_var'] == column]['type'].values[0] == 'mean':
                    election_data[column] = aggregation.weighted_amean(valid_values, valid_tract_pops)
                elif statistics[statistics['agg_var'] == column]['type'].values[0] in ['percentage', 'ratio']:
                    election_data[column] = aggregation.weighted_amean(valid_values / 100, valid_tract_pops) * 100
                elif statistics[statistics['agg_var'] == column]['type'].values[0] == 'median':
                    election_data[column] = aggregation.weighted_gmean(valid_values, valid_tract_pops)
            else:
                # if every value is not nan
                if any(~np.isnan(values)):
                    pops = tract_rows['DP05_0001E'].values
                    percents = tract_rows['percent'].values
                    tract_pops = pops*percents
                    # get the values in values that are not NaN as well as the corresponding values in DP05_0001E and the corresponding percents
                    valid_pops = pops[~np.isnan(values)]
                    valid_tract_pops = tract_pops[~np.isnan(values)]

                    props = valid_values / valid_pops
                    weighted_prop = aggregation.weighted_amean(props, valid_tract_pops)

                    # replace the Nan values in values with tract_pop * weighted_prop
                    for i in range(len(values)):
                        if np.isnan(values[i]):
                            values[i] = pops[i] * weighted_prop

                election_data[column] = aggregation.weighted_sum(values, percents)

    all_data.append(election_data)

0 of 356 rows processed
10 of 356 rows processed
20 of 356 rows processed
30 of 356 rows processed
40 of 356 rows processed
50 of 356 rows processed
60 of 356 rows processed
70 of 356 rows processed
80 of 356 rows processed
90 of 356 rows processed
100 of 356 rows processed
110 of 356 rows processed
120 of 356 rows processed
130 of 356 rows processed
140 of 356 rows processed
150 of 356 rows processed
160 of 356 rows processed
170 of 356 rows processed
180 of 356 rows processed
190 of 356 rows processed
200 of 356 rows processed
210 of 356 rows processed
220 of 356 rows processed
230 of 356 rows processed
240 of 356 rows processed
250 of 356 rows processed
260 of 356 rows processed
270 of 356 rows processed
280 of 356 rows processed
290 of 356 rows processed
300 of 356 rows processed
310 of 356 rows processed
320 of 356 rows processed
330 of 356 rows processed
340 of 356 rows processed
350 of 356 rows processed


In [85]:
# save missing_data to csv
missing_data.to_csv('missing_data.csv', index=False)

In [86]:
all_data_df = pd.DataFrame(all_data)
result_df = pd.concat([elections_with_tracts, all_data_df], axis=1)
result_df

,filename,District #,tracts,percents,num_tracts,DP02_0130E,DP03_0051E,DP03_0010E,DP03_0121E,DP02_0090E,...,DP02_0134E,DP05_0050E,DP02_0007E,DP05_0037E,DP02_0057E,DP03_0115E,DP02_0062E,DP03_0067E,DP05_0003E,DP03_0063E
0,Berkeley_11022010_CityCouncilDistrict1.csv,1,"06_001_422200, 06_001_422400, 06_001_422300, 0...","0.9963084989162369, 0.0011620138428508448, 0.7...",11,32.794571,5773.893958,5686.092603,NaN,4121.914667,...,1281.640671,34.625574,70.305388,0.000000e+00,1844.936555,1407.098343,1193.711815,14575.898761,6670.735851,85367.256101
1,Berkeley_11042014_CityCouncilDistrict1.csv,1,"06_001_422200, 06_001_422400, 06_001_422300, 0...","0.9937216895775107, 0.007068638715127452, 0.99...",13,57.256150,6431.604448,6466.142331,NaN,4933.928948,...,1387.387008,11.824393,79.474016,0.000000e+00,1952.043174,1846.596519,1641.762744,16348.623314,7470.607088,94607.202016
2,Berkeley_11062018_CityCouncilDistrict1.csv,1,"06_001_422200, 06_001_422400, 06_001_422300, 0...","0.9937216895775107, 0.007068638715127452, 0.99...",13,1488.112721,6315.832053,6670.277986,NaN,12116.037900,...,97.933612,85.100078,43.860910,9.074205e+03,668.407670,1832.335565,868.350401,16835.947025,7854.397387,136565.780094
3,Minneapolis_11022021_CityCouncilWard1.csv,1,"27_053_100400, 27_053_100500, 27_053_100900, 2...","2.65672307536351e-10, 0.9999999644942321, 3.91...",27,2213.179872,15028.612188,13658.924964,NaN,29283.983388,...,138.357805,199.182536,183.502428,2.428620e+04,1117.109868,2720.870617,4079.446340,18003.036561,15766.381640,100209.535579
4,Minneapolis_11052013_Ward13CityCouncil.csv,1,"27_123_041103, 27_123_042002, 27_123_041301, 2...","9.332052281831136e-18, 6.283329947436996e-06, ...",25,286.206231,14353.387198,14086.730976,NaN,9383.256630,...,4142.934852,0.000000,319.752528,2.241221e-14,4404.212117,3626.439960,5115.663741,15761.693757,16876.868822,66995.454164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,Alaska_11082022_SenateDistrictP.csv,P,"02_090_000100, 02_090_000300, 02_090_000400, 0...","1.0000000000000002, 1.0000000000000004, 1.0000...",15,1826.615527,10605.279625,9597.146957,NaN,22615.642296,...,19.920950,4.804059,149.988888,1.550206e+04,1278.247103,2653.906320,5264.023611,18237.063478,11865.136253,88133.238751
352,Alaska_11082022_SenateDistrictQ.csv,Q,"02_240_000100, 02_090_001700, 02_090_001300, 0...","4.155651760364181e-07, 0.9756357325060139, 0.7...",16,4197.819067,12475.472919,12503.010682,NaN,33706.995113,...,58.024024,110.401748,266.417212,2.788346e+04,1884.414457,4045.924793,5138.713188,20896.004266,16343.943878,124300.851344
353,Alaska_11082022_SenateDistrictR.csv,R,"02_240_000100, 02_170_000101, 02_180_000100, 0...","0.999998499378834, 6.169030906028319e-07, 2.70...",32,4088.812079,17962.942383,16723.280072,NaN,45052.613032,...,18.550638,39.797624,177.184629,3.295309e+04,2166.398135,5688.554142,8566.662428,19424.336582,22372.501922,99037.317495
354,Alaska_11082022_SenateDistrictS.csv,S,"02_170_000101, 02_013_000100, 02_050_000200, 0...","6.101021175696617e-08, 0.4775048649755406, 0.9...",16,401.934745,7447.485427,9214.452532,NaN,26754.731799,...,2.448476,78.840707,132.136166,3.897210e+03,2022.381150,3005.821446,7791.620247,17062.885135,13214.720058,87739.136515


In [87]:
# save result_df as final_data.csv
result_df.to_csv('final_data.csv', index=False)